## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [4]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable


os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000262F75383A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000262F7539030>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Harshal and I am a Gen AI Engineer")])

AIMessage(content="Hi Harshal,\n\nIt's nice to meet you! Being a Gen AI Engineer is an exciting field to be in. \n\nWhat kind of projects are you working on? I'm always interested in learning more about the innovative things people are doing with generative AI.\n", response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 23, 'total_tokens': 82, 'completion_time': 0.107272727, 'prompt_time': 0.000169369, 'queue_time': 0.013638119999999998, 'total_time': 0.107442096}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ef79ebcf-5f0e-4c8f-b15f-ad0676cb586e-0', usage_metadata={'input_tokens': 23, 'output_tokens': 59, 'total_tokens': 82})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Harshal and I am a Gen AI Engineer"),
        AIMessage(content="Hello Harshal! It's nice to meet you. \n\nAs a Gen AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Harshal and that you are a Gen AI Engineer!  \n\nIs there anything else you'd like to tell me about your work? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 99, 'total_tokens': 137, 'completion_time': 0.069090909, 'prompt_time': 0.006064197, 'queue_time': 0.054743731, 'total_time': 0.075155106}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-43531426-15bd-40a4-9f32-42369d6d5200-0', usage_metadata={'input_tokens': 99, 'output_tokens': 38, 'total_tokens': 137})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Harshal and I am a Gen AI Engineer")],
    config=config
)

In [11]:
response.content

"Hi Harshal, it's nice to meet you! \n\nThat's awesome! Being a Gen AI Engineer is a really exciting field to be in. What kind of projects are you working on? \n\nI'm always interested to learn more about what people are doing with generative AI.  😊  \n\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Harshal,  as you told me when you introduced yourself! 😊 \n\n\n', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 104, 'total_tokens': 125, 'completion_time': 0.038181818, 'prompt_time': 0.005301189, 'queue_time': 0.00808346, 'total_time': 0.043483007}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b142a087-6d77-4a96-85ac-e6c75b856bdd-0', usage_metadata={'input_tokens': 104, 'output_tokens': 21, 'total_tokens': 125})

In [13]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊 \n\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hello John! 👋  It's nice to meet you. \n\nIs there anything I can help you with today? 😊  \n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  I remember you told me earlier. 😊  \n\nHow can I help you today, John? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Harshal")]})

AIMessage(content='Nice to meet you, Harshal! How can I help you today? 😊 \n', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 31, 'total_tokens': 51, 'completion_time': 0.036363636, 'prompt_time': 0.000345008, 'queue_time': 0.015201052, 'total_time': 0.036708644}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-693ebb08-b7aa-48b8-aa48-1cb459908276-0', usage_metadata={'input_tokens': 31, 'output_tokens': 20, 'total_tokens': 51})

In [18]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Harshal")],
    config=config
)

response

AIMessage(content="Hi Harshal! It's nice to meet you.  \n\nWhat can I do for you today? 😊  I'm ready to answer your questions and help with any tasks you have. \n\n", response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.003687145, 'queue_time': 0.043127479999999996, 'total_time': 0.085505327}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e1e716c-4db6-4340-890f-03cdad0801da-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Harshal!  😊  I remember that you told me.  \n\nIs there anything else I can help you with?'

In [21]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Harshal")],"language":"Hindi"})
response.content

'नमस्ते हर्षल! \n\nमुझे बहुत अच्छा लग रहा है कि आपने मुझसे बात करना चुना। मैं आपकी मदद करने को तैयार हूँ।  \n\nआप क्या पूछना चाहते हैं? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [23]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [24]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Harshal")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते हर्षल!  \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आप मुझसे क्या पूछना चाहेंगे? 😊 \n'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [26]:
response.content

'आपका नाम हर्षल है! 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

**trim_messages** helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"